In [1]:
import os
import re
import pandas as pd

In [2]:
exp_folder = "experiments/"
dirfiles = os.listdir(exp_folder)

fullpaths = map(lambda name: os.path.join(exp_folder, name), dirfiles)

dirs = []
files = []

for file in fullpaths:
    if os.path.isdir(file): dirs.append(file)
    if os.path.isfile(file): files.append(file)

print(list(dirs))
#print(list(files))

['experiments/ai_pretrain_then_finetune', 'experiments/drugs_pretrain_then_finetune', 'experiments/transfer']


In [4]:
#dirs=['/home/bogenspergerj/master/CrossNERV2/CrossNER/experiments/transfer/12']

In [5]:
dev_results={}
test_results={}

for folder in dirs:
    log_file_path = folder + '/train.log'
    exp_id=log_file_path.split('/')[-2]
    with open(log_file_path) as f:
        lines = [line.rstrip() for line in f]
        #print(exp_id)
        model_name=''
        cut_strategy=''
        dropout=''
        dapt=''
        highest_f1=0
        highest_f1_test=0
        for line in lines:
            if 'model_name' in line:
                match = re.search('model_name: ', line)
                model_name=line[match.end():].rstrip()
                
            if 'cut_strategy: ' in line:
                match = re.search('cut_strategy: ', line)
                cut_strategy=line[match.end():].rstrip()
                
            if 'dropout: ' in line:
                match = re.search('dropout: ', line)
                dropout=line[match.end():].rstrip()
                
            if 'ckpt: ' in line:
                match = re.search('ckpt: ', line)
                ckpt=line[match.end():].rstrip()
                dapt = ckpt.split('/')[2]
                
            if 'Evaluate on Test Set.' in line:
                if dapt == '': dapt='None'
                try:
                    f1_txt="0"
                    result = line.split('F1: ')[1]#get part after F1:
                    f1_txt = result.split('. Prec.: ')[0]#
                    result = result.split('. Prec.: ')[1]
                    prec_txt = result.split('. Rec.: ')[0]
                    rec_txt = result.split('. Rec.: ')[1][:-1]# remove the last dot in the end of the recall..
                except IndexError:
                    print('INDEXERROR at line: ', line, 'in file: ', log_file_path) # pre-training test evaluation has only f1 score
                    #so actually we are happy that this causes an error if we wanna extract those results
                    f1_txt="0"
                    continue
                
                #only store higher f1 values..
                new_f1 = float(f1_txt)
                if new_f1 >= highest_f1_test:
                    print('New F1:', new_f1)
                    highest_f1_test = new_f1
                    
                    #german use german style comma separator and not dot style
                    f1_txt = f1_txt.replace('.', ',')
                    prec_txt = prec_txt.replace('.', ',')
                    rec_txt = rec_txt.replace('.', ',')
                    test_results[int(exp_id)]=[model_name, dapt, dropout, cut_strategy, 
                                  f1_txt, prec_txt, rec_txt]#Only keep last--> therefore overwrite previous
            
            if 'Evaluate on Dev Set.' in line:
                
                if dapt == '': dapt='None'
                try:
                    
                    f1_txt=""
                    result = line.split('F1: ')[1]#get part after F1:
                    f1_txt = result.split('. Prec.: ')[0]#
                    result = result.split('. Prec.: ')[1]
                    prec_txt = result.split('. Rec.: ')[0]
                    rec_txt = result.split('. Rec.: ')[1][:-1]# remove the last dot in the end of the recall..
                    
                    
                    
                except IndexError:
                    print('INDEXERROR at line: ', line, 'in file: ', log_file_path) # pre-training test evaluation has only f1 score
                    #so actually we are happy that this causes an error if we wanna extract those results
                    f1_txt="0"
                    continue
                
                #only store higher f1 values..
                new_f1 = float(f1_txt)
                print('check F1:', new_f1, ' highest F1', highest_f1)
                if new_f1 >= highest_f1:
                    print('New F1:', new_f1)
                    highest_f1 = new_f1
                    
                    #german use german style comma separator and not dot style
                    f1_txt = f1_txt.replace('.', ',')
                    prec_txt = prec_txt.replace('.', ',')
                    rec_txt = rec_txt.replace('.', ',')
                    dev_results[int(exp_id)]=[model_name, dapt, dropout, cut_strategy, 
                                      f1_txt, prec_txt, rec_txt]#Only keep last--> therefore overwrite previous

NameError: name 'dirs' is not defined

In [6]:
pd.options.display.float_format = '${:,.2f}'.format

# Dev Results Exploration

In [7]:
dev_result_table = pd.DataFrame.from_dict(dev_results, orient='index')
dev_result_table.columns=['model', 'dapt', 'dropout', 'cut_strategy',  'f1', 'precision', 'recall']

pd.set_option('display.max_rows', None)
dev_result_table.sort_index(inplace=True)
dev_result_table

ValueError: Length mismatch: Expected axis has 0 elements, new values have 7 elements

In [8]:
sorted_dev_df = dev_result_table.sort_values(by='f1', ascending=False)
sorted_dev_df #[sorted_df['dapt']=='None']

KeyError: 'f1'

# Test Dataset Results

In [18]:
test_result_table = pd.DataFrame.from_dict(test_results, orient='index')
test_result_table.columns=['model', 'dapt', 'dropout', 'cut_strategy',  'f1', 'precision', 'recall']

pd.set_option('display.max_rows', None)
test_result_table.sort_index(inplace=True)
#test_result_table

In [9]:
sorted_test_df = test_result_table.sort_values(by='f1', ascending=False)
#sorted_test_df #[sorted_df['dapt']=='None']

# Explore the results

In [10]:
numeric_results = test_result_table.copy()
numeric_results['f1'] = numeric_results['f1'].str.replace(',', '.')
numeric_results['precision'] = numeric_results['precision'].str.replace(',', '.')
numeric_results['recall'] = numeric_results['recall'].str.replace(',', '.')

numeric_results['f1'] = numeric_results['f1'].astype(float)
numeric_results['precision'] = numeric_results['precision'].astype(float)
numeric_results['recall'] = numeric_results['recall'].astype(float)

### Average Scores per LM

In [11]:
numeric_results[['model', 'f1', 'precision', 'recall']].groupby('model').mean('f1')

,f1,precision,recall
model,,,
bert-base-cased,$82.40,$84.26,$80.64
roberta-base,$82.23,$84.09,$80.51


### Max Score per LM

In [12]:
numeric_results[['model', 'f1', 'precision', 'recall']].groupby('model').max('f1')

,f1,precision,recall
model,,,
bert-base-cased,$83.85,$87.26,$83.23
roberta-base,$84.18,$87.15,$84.45


### Avg Scores per DAPT

In [13]:
numeric_results[['dapt', 'f1', 'precision', 'recall']].groupby('dapt').mean('f1')

,f1,precision,recall
dapt,,,
All,$83.06,$84.92,$81.31
Dreammarket,$82.21,$83.96,$80.57
Grams,$82.32,$84.16,$80.60
None,$81.69,$83.26,$80.24
Wiki,$82.28,$84.57,$80.16


### Max Scores per DAPT

In [14]:
numeric_results[['dapt', 'f1', 'precision', 'recall']].groupby('dapt').max('f1')

,f1,precision,recall
dapt,,,
All,$84.18,$87.26,$84.45
Dreammarket,$83.54,$86.76,$83.01
Grams,$83.59,$86.41,$83.13
None,$83.36,$85.73,$83.23
Wiki,$83.67,$87.15,$83.41


### Avg Scores per Dropout

In [15]:
numeric_results[['dropout', 'f1', 'precision', 'recall']].groupby('dropout').mean('f1')

,f1,precision,recall
dropout,,,
0.0,$82.15,$83.38,$81.00
0.05,$82.33,$84.53,$80.27
0.1,$82.42,$84.48,$80.51
0.15,$82.25,$84.71,$79.94
0.2,$82.37,$84.01,$80.84
0.25,$82.33,$84.32,$80.44
0.3,$81.99,$83.84,$80.28
0.35,$82.12,$83.99,$80.38
0.4,$82.43,$84.56,$80.48


### Max Scores per Dropout

In [16]:
numeric_results[['dropout', 'f1', 'precision', 'recall']].groupby('dropout').max('f1')

,f1,precision,recall
dropout,,,
0.0,$83.66,$85.99,$83.59
0.05,$83.32,$86.41,$81.88
0.1,$84.18,$86.30,$82.79
0.15,$83.46,$85.94,$82.90
0.2,$83.49,$86.44,$83.13
0.25,$83.51,$85.29,$82.16
0.3,$83.67,$86.76,$83.01
0.35,$83.43,$85.85,$82.44
0.4,$83.59,$87.26,$83.41


# Store Results in folder

In [17]:
result_table.to_csv(exp_folder+'results.csv', sep=';')

NameError: name 'result_table' is not defined

In [ ]:
#Convert to latex code
#result_table.to_latex()